### Применение индексов

In [ ]:
# импортируем numpy и pandas
import numpy as np
import pandas as pd

# импортируем datetime
import datetime
from datetime import datetime, date

In [ ]:
# создаем DataFame, состоящий из случайных чисел и столбца key
df = pd.DataFrame({'foo':np.random.random(10000), 'key':range(100, 10100)})
df[:5]

In [ ]:
# отбираем строку, в которой значение столбца key равно 10099
df[df.key==10099]

In [ ]:
# измеряем время выполнения операции отбора
%timeit df[df.key==10099]

In [ ]:
# превращаем столбец key в index
df_with_index = df.set_index(['key'])
df_with_index[:5]

In [ ]:
# теперь можно найти это значение с помощью индекса
df_with_index.loc[10099]

In [ ]:
# и теперь операция выполняется намного быстрее
%timeit df_with_index.loc[10099]

### Основной тип Index

In [ ]:
# покажем, что столбцы фактически являются индексом
temps = pd.DataFrame({ "City": ["Москва", "Петербург"],
                       "Температура": [3, 1] })
temps

In [ ]:
# мы видим, что столбцы - это индекс
temps.columns

### Индексы Int64Index и RangeIndex, в качестве меток используются целые числа

In [ ]:
# явно создаем Int64Index
df_i64 = pd.DataFrame(np.arange(10, 20), index=np.arange(0, 10))
df_i64[:5]

In [ ]:
# смотрим индекс
df_i64.index

In [ ]:
# по умолчанию мы получаем RangeIndex
df_range = pd.DataFrame(np.arange(10, 15))
df_range[:5]

In [ ]:
df_range.index

### Индекс Float64Index, в качестве меток используются числа с плавающей точкой

In [ ]:
# индексы, использующие Float64Index
df_f64 = pd.DataFrame(np.arange(0, 1000, 5), np.arange(0.0, 100.0, 0.5))
df_f64.iloc[:5] # потребуется iloc для отбора первых пяти строк

In [ ]:
df_f64.index

### Индексирование по датам и времени с помощью DatetimeIndex

In [ ]:
# создаем DatetimeIndex на основе диапазона дат
rng = pd.date_range('5/1/2017', periods=5, freq='H')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

In [ ]:
ts.index

### Индексирование периодов времени с помощью PeriodIndex

In [ ]:
# явно создаем PeriodIndex
periods = pd.PeriodIndex(['2017-1', '2017-2', '2017-3'], freq='M')
periods

In [ ]:
period_series = pd.Series(np.random.randn(len(periods)), 
                          index=periods)  # используем индекс в серии
period_series

### Создание и использование индекса в объекте Series или объекте DataFrame

In [ ]:
# создаем DatetimeIndex
date_times = pd.DatetimeIndex(pd.date_range('5/1/2017', 
                                            periods=5, 
                                            freq='H'))
date_times

In [ ]:
# создаем объект DataFrame, используя индекс
df_date_times = pd.DataFrame(np.arange(0, len(date_times)), 
                             index=date_times)
df_date_times

In [ ]:
# задаем индекс датафрейма
df_date_times.index = pd.DatetimeIndex(pd.date_range('6/1/2017', 
                                                     periods=5, 
                                                     freq='H'))
df_date_times

### Отбор значений с помощью индекса

In [ ]:
# создаем серию
s = pd.Series(np.arange(0, 5), index=list('abcde'))
s

In [ ]:
# ищем по метке индекса
s['b']

In [ ]:
# явно ищем по метке индекса
s.loc['b']

In [ ]:
# создаем датафрейм с двумя столбцами
df = pd.DataFrame([ np.arange(10, 12), 
                    np.arange(12, 14)], 
                  columns=list('ab'), 
                  index=list('vw'))
df

In [ ]:
df['a'] # эта строка возвращает столбец 'a'

In [ ]:
df.loc['w'] # возвращает строку 'w' по метке

In [ ]:
s['b':'d'] # создаем срез серии от метки b до метки d

In [ ]:
s.loc['b':'d'] # эта строка явно создает срез от метки b до метки d

In [ ]:
# ищем строки по метке
s.loc[['a', 'c', 'e']]

### Преобразование данных в индекс и получение данных из индекса 

In [ ]:
orders = pd.read_csv("Data/orders.csv", 
                    index_col='id')

orders.head()

In [ ]:
# сбрасываем индекс, помещая значения индекса в столбец
index_moved_to_col = orders.reset_index()
index_moved_to_col[:5]

In [ ]:
# а теперь делаем столбец id индексом
index_moved_to_col.set_index('id')[:5]

### Иерархическая индексация

In [ ]:
# сначала помещаем символы в столбец
reindexed = orders.reset_index()
# а теперь индексируем датафрейм 
multi_fi = reindexed.set_index(['order_date', 'customer_id'])
multi_fi[:5]

In [ ]:
# наш индекс - это MultiIndex
type(multi_fi.index)

In [ ]:
# он имеет два уровня
len(multi_fi.index.levels)

In [ ]:
# каждый уровень индекса - это индекс
multi_fi.index.levels[0]

In [ ]:
# каждый уровень индекса - это индекс
multi_fi.index.levels[1]

In [ ]:
# значения в уровне индекса 0
multi_fi.index.get_level_values(0)

In [ ]:
multi_fi.xs('2014-03-14')[:5]

In [ ]:
# отбираем строки, в которых индекс уровня 0 
# имеет значение 2014-03-14, без удаления уровней
multi_fi.xs('2014-03-14', drop_level=False)[:5]

In [ ]:
# скомбинируем уровни индексов
multi_fi.xs('2014-03-14').xs('NF-18475')